<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jcc/Train_Test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cargamos el archivo con las consultas que está en Github
from os import path

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists('03-Correos_variables_estaticas.csv')):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/03-Correos_variables_estaticas.csv

# Leemos el archivo en un dataframe
import pandas as pd

df = pd.read_csv('03-Correos_variables_estaticas.csv', delimiter="|") 

In [2]:
# Limpio el campo proveedor_correo

df['proveedor_correo'].replace({"autloock": "outlook",
                                "outlool": "outlook",
                                "gmai": "gmail",
                                "gmal": "gmail",
                                "gmil": "gmail",
                                "hotmai": "hotmail",
                                "luve": "live",
                                }, inplace=True)

# Se transforma proveedor_correo a numerico
from sklearn import preprocessing
le_correo = preprocessing.LabelEncoder()
df['proveedor_correo'] = le_correo.fit_transform(df['proveedor_correo'])

le_correo.classes_

array(['ampinformatica', 'baires-sa', 'correoargentino', 'facebook',
       'gmail', 'grupoberro', 'hotmail', 'icloud', 'live', 'me', 'msn',
       'outlook', 'uno', 'yahoo'], dtype=object)

In [3]:
# Me guardo los atributos, excepto la clase en x

x = df.drop(['Clase'], axis=1)
x.columns

Index(['Consulta', 'dia_semana', 'semana_del_mes', 'mes', 'cuatrimestre',
       'anio', 'hora_discretizada', 'dni_discretizado', 'legajo_discretizado',
       'posee_legajo', 'posee_telefono', 'carrera_valor', 'proveedor_correo',
       'cantidad_caracteres', 'proporcion_mayusculas', 'proporcion_letras',
       'cantidad_tildes', 'cantidad_palabras', 'cantidad_palabras_cortas',
       'proporcion_palabras_distintas', 'frecuencia_signos_puntuacion',
       'cantidad_oraciones', 'utiliza_codigo_asignatura'],
      dtype='object')

In [4]:
# Se numeriza la clase
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# Me quedo con las clases numerizadas
y=le.fit_transform(df['Clase'])

# Por otro lado me guardo las etiquetas de las clases
target_names=le.classes_

target_names

array(['Boleto Universitario', 'Cambio de Carrera', 'Cambio de Comisión',
       'Carga de Notas', 'Certificados Web', 'Consulta por Equivalencias',
       'Consulta por Legajo', 'Consulta sobre Título Universitario',
       'Cursadas', 'Datos Personales', 'Exámenes',
       'Ingreso a la Universidad', 'Inscripción a Cursadas',
       'Pedido de Certificados', 'Problemas con la Clave',
       'Reincorporación', 'Requisitos de Ingreso',
       'Simultaneidad de Carreras', 'Situación Académica',
       'Vacunas Enfermería'], dtype=object)

In [5]:
# Separo datos de entrenamiento y testing
from sklearn.model_selection import train_test_split

# Separo en 80-20 entrenamiento/validación y testeo
df_train, df_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.2, stratify=y)

In [6]:
# Reordeno el índice para poder concatenar con el y_train

# Train
df_train.reset_index(drop=True, inplace=True)
y_train_series = pd.Series(y_train)
df_train.merge(y_train_series.rename('clase'), left_index=True, right_index=True)

# Test
df_test.reset_index(drop=True, inplace=True)
y_test_series = pd.Series(y_test)
df_test.merge(y_test_series.rename('clase'), left_index=True, right_index=True)

,Consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura,clase
0,buen día. me llegó un mail que ya está habilit...,2,2,3,1,2019,0,5,4,1,0,52,6,170,0.0,0.782353,5,34,24,0.911765,0.023529,3,0,0
1,"no puedo acceder, probablemente sea por poner ...",3,4,11,2,2016,0,5,3,1,1,27,4,127,0.0,0.818898,0,16,7,1.000000,0.047244,3,0,16
2,"buenas tardes, me dicen que tengo que reincri...",1,4,2,1,2017,2,1,1,1,1,5,13,332,0.0,0.801205,0,63,35,0.746032,0.012048,3,0,9
3,hola quería saber si recibieron la solicitud d...,5,1,2,1,2019,0,2,0,0,1,0,6,165,0.0,0.806061,3,29,18,0.862069,0.018182,3,0,11
4,quisiera obtener la regularidad para poder ren...,4,1,3,1,2019,1,2,1,1,1,54,6,93,0.0,0.838710,0,14,5,1.000000,0.021505,1,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,no puedo reseterar mi clave de acceso al siste...,4,2,12,2,2015,0,2,1,1,1,5,6,419,0.0,0.809069,0,78,45,0.717949,0.007160,1,0,14
196,"buenas tardes, quería saber por qué motivo las...",4,1,9,2,2019,1,8,4,1,1,54,6,168,0.0,0.797619,4,30,16,0.933333,0.029762,2,0,0
197,"hola, me olvide el numero de contraseña de mi ...",4,1,8,2,2017,1,3,1,1,1,3,4,84,0.0,0.809524,0,15,7,0.866667,0.023810,1,0,14
198,corregir datos erróneos sobre mi domicilio. do...,3,2,3,1,2016,3,3,3,1,1,52,11,241,0.0,0.763485,3,37,14,0.945946,0.020747,2,0,9


In [7]:
# Me autentico en Drive
from google.colab import drive

# Guardo en Drive los datos de entrenamiento
df_train.to_csv('correos-train-80.csv', index=False)
drive.mount('drive')
!cp correos-train-80.csv "drive/My Drive/"

# Guardo en Drive los datos de testeo
df_test.to_csv('correos-test-20.csv', index=False)
drive.mount('drive')
!cp correos-test-20.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
